In [1]:
%matplotlib inline
%run ../../import_envs.py
print('probtorch:', probtorch.__version__, 
      'torch:', torch.__version__, 
      'cuda:', torch.cuda.is_available())

probtorch: 0.0+5a2c637 torch: 0.4.1 cuda: True


In [2]:
## Load dataset
data_path = "../gmm_dataset_3c"
Data = torch.from_numpy(np.load(data_path + '/obs.npy')).float()

NUM_DATASETS, N, D = Data.shape
K = 3 ## number of clusters
SAMPLE_SIZE = 10
NUM_HIDDEN_LOCAL = 32

MCMC_SIZE = 10
BATCH_SIZE = 20
NUM_EPOCHS = 200
LEARNING_RATE = 5 * 1e-4
CUDA = torch.cuda.is_available()
PATH = 'ag-joint-both'
DEVICE = torch.device('cuda:1')

Train_Params = (NUM_EPOCHS, NUM_DATASETS, SAMPLE_SIZE, BATCH_SIZE, CUDA, DEVICE, PATH)
Model_Params = (N, K, D, MCMC_SIZE)

In [3]:
from local_enc import *
from global_oneshot import *
from global_enc_v1 import *
## if reparameterize continuous variables
Reparameterized = False
# initialization
enc_z = Enc_z(K, D, NUM_HIDDEN_LOCAL, CUDA, DEVICE)
enc_eta = Enc_eta(K, D, CUDA, DEVICE, Reparameterized)
oneshot_eta = Oneshot_eta(K, D, CUDA, DEVICE, Reparameterized)

if CUDA:
    enc_z.cuda().to(DEVICE)
    enc_eta.cuda().to(DEVICE)
    oneshot_eta.cuda().to(DEVICE)
optimizer =  torch.optim.Adam(list(oneshot_eta.parameters())+list(enc_eta.parameters())+list(enc_z.parameters()),lr=LEARNING_RATE, betas=(0.9, 0.99))
models = (oneshot_eta, enc_eta, enc_z)

In [ ]:
from ag_ep import *
train(models, EUBO_init_eta_joint_both, optimizer, Data, Model_Params, Train_Params)

epoch: 0\200 (132s),  loss: -12609.133,  ess: 1.055,  kl_eta_ex: 368.086,  kl_eta_in: 197.540,  kl_z_ex: 76.198,  kl_z_in: 20.801
epoch: 1\200 (135s),  loss: -9665.123,  ess: 1.119,  kl_eta_ex: 185.048,  kl_eta_in: 285.846,  kl_z_ex: 20.463,  kl_z_in: 9.919
epoch: 2\200 (133s),  loss: -9352.736,  ess: 1.127,  kl_eta_ex: 165.544,  kl_eta_in: 314.056,  kl_z_ex: 15.218,  kl_z_in: 8.083
epoch: 3\200 (136s),  loss: -9200.280,  ess: 1.129,  kl_eta_ex: 157.347,  kl_eta_in: 299.606,  kl_z_ex: 13.749,  kl_z_in: 7.577
epoch: 4\200 (134s),  loss: -9055.847,  ess: 1.123,  kl_eta_ex: 150.005,  kl_eta_in: 291.199,  kl_z_ex: 12.842,  kl_z_in: 7.299
epoch: 5\200 (134s),  loss: -8936.190,  ess: 1.126,  kl_eta_ex: 143.837,  kl_eta_in: 284.160,  kl_z_ex: 12.650,  kl_z_in: 7.278
epoch: 6\200 (135s),  loss: -8832.367,  ess: 1.120,  kl_eta_ex: 138.833,  kl_eta_in: 288.417,  kl_z_ex: 12.495,  kl_z_in: 7.164
epoch: 7\200 (135s),  loss: -8741.227,  ess: 1.119,  kl_eta_ex: 136.079,  kl_eta_in: 297.323,  kl_z_ex

In [ ]:
torch.save(enc_z.state_dict(), "../weights/enc-z-%s" % PATH)
torch.save(enc_eta.state_dict(), "../weights/enc-eta-%s" % PATH)
torch.save(oneshot_eta.state_dict(), "../weights/oneshot-eta-%s" % PATH)

In [ ]:
indices = torch.arange(NUM_DATASETS)
step = 2
batch_indices = indices[step*BATCH_SIZE : (step+1)*BATCH_SIZE]
obs = Data[batch_indices]
obs = shuffler(obs).repeat(SAMPLE_SIZE, 1, 1, 1)
if CUDA:
    obs =obs.cuda().to(DEVICE)
    
q_eta, p_eta, q_nu = oneshot_eta(obs, K, D)
obs_mu = q_eta['means'].value

In [ ]:
obs_mu[0,0]

In [ ]:
q_eta['means'].dist.loc[0]